# Constrained Tikhonov regularization implemented by semismooth Newton method
We consider the two-dimensional deconvolution problems to find a non-negative function f given data 
$$
    d \sim \mathrm{Pois}(h*f)
$$
with a non-negative convolution kernel $h$, and $\mathrm{Pois}$ denotes the element-wise Poisson distribution.

We explore the use of the semismooth Newton method to implement constrained Tikhonov regularization 
$$
\hat{f} = \mathrm{argmin}_{f\geq 0} \left[\| h*f-d\|^2_{L^2(w)} + \alpha \|f\|^2_{L^2}\right]
$$
with a weight $w = \frac{1}{\sqrt{d+1}}$. The regularization parameter $\alpha$ is chosen by the discrepancy principle.   

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mplib 
from regpy.operators.convolution import GaussianBlur
from regpy.vecsps import UniformGridFcts
from regpy.solvers import RegularizationSetting
from regpy.solvers.linear.semismoothNewton import SemismoothNewton_bilateral, SemismoothNewton_nonneg, SemismoothNewtonAlphaGrid
from regpy.hilbert import L2
from regpy.stoprules import CountIterations, Discrepancy
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(name)-20s :: %(message)s'
)

### plotting routine for comparing reconstructions and originals 

In [ ]:
def comparison_plot(grid,truth,reco,title_right='exact',title_left='reconstruction',residual=None):
    plt.rcParams.update({'font.size': 22})
    extent = [grid.axes[0][0],grid.axes[0][-1], grid.axes[1][0], grid.axes[1][-1]]
    maxval = np.max(truth[:]); minval = np.min(truth[:])
    mycmap = mplib.colormaps['hot']
    mycmap.set_over((0,0,1.,1.))  # use blue as access color for large values
    mycmap.set_under((0,1,0,1.))  # use green as access color for small values
    if not (residual is None):
        fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(2,2,figsize = (22,16))
    else:
        fig, (ax1,ax2) = plt.subplots(1,2,figsize = (22,8))
    im1= ax1.imshow(reco.T,extent=extent,origin='lower',
                    vmin=1.05*minval-0.05*maxval, vmax =1.05*maxval-0.05*minval,
                    cmap=mycmap
                    )
    ax1.title.set_text(title_left)
    fig.colorbar(im1,extend='both')
    im2= ax2.imshow(truth.T,extent=extent, origin='lower',
                    vmin=1.05*minval-0.05*maxval, vmax =1.05*maxval-0.05*minval,
                    cmap=mycmap
                    )
    ax2.title.set_text(title_right)
    fig.colorbar(im2,extend='both',orientation='vertical')
    if not (residual is None):
        maxv = np.max(reco[:]-truth[:])
        im3 = ax3.imshow(truth.T-reco.T,extent=extent, origin='lower',vmin= -maxv,vmax=maxv,cmap='RdYlBu_r')
        ax3.title.set_text('reconstruction error')
        fig.colorbar(im3)

        maxv = np.max(residual[:])
        im4 = ax4.imshow(residual.T,extent=extent, origin='lower',vmin= -maxv,vmax=maxv,cmap='RdYlBu_r')
        ax4.title.set_text('data residual')
        fig.colorbar(im4)

### test objects

In [ ]:
grid = UniformGridFcts((-1, 1, 256), (-1.5, 1, 256),dtype = float, periodic = True)
"""Space of real-valued functions on a uniform grid with rectangular pixels"""
X = grid.coords[0]; Y = grid.coords[1]
"""x and y coordinates."""
cross = 1.0*np.logical_or((abs(X)<0.01) * (abs(Y)<0.3),(abs(X)<0.3) * (abs(Y)<0.01)) 
rad = np.sqrt(X**2 + Y**2)
ring = 1.0*np.logical_and(rad>=0.9, rad<=0.95)
smallbox = (abs(X+0.55)<=0.05) * (abs(Y-0.55)<=0.05)
bubbles = (1.001+np.sin(50/(X+1.3)))*np.exp(-((Y+1.25)/0.1)**2)*(X>-0.8)*(X<0.8)

objects = 200*(ring + 2.0*cross + 1.5*smallbox + bubbles)
exact_sol = objects

### creating Poisson distributed synthetic data

In [ ]:
a=0.15
conv = GaussianBlur(grid,a,pad_amount=((16,16),(16,16)))
"""Convolution operator \(f\mapsto h*f\) for the convolution kernel \(h(x)=\exp(-|x|_2^2/a^2)\)."""
#conv = ExponentialConvolution(grid,a)
"""Convolution operator \(f\mapsto h*f\) for the convolution kernel \(h(x)=\exp(-|x|_1/a)\)."""

blur = conv(exact_sol)
blur[blur<0] = 0.
"""Simulated exact data."""
data = np.random.poisson(blur)
"""Simulated measured data. The Poisson distribution occurs if photon count detectors are used."""
comparison_plot(grid,exact_sol,data,title_left='noisy measurement data')

### Convergence of the semismooth Newton method
The most appropriate method is ``SemismoothNewton_nonneg`` implementing the constraint $f\geq 0$. The same results are obtained by 
the more general method ``SemismoothNewton_bilateral`` implementing bilateral constaints $\psi_-\leq f\leq \psi_+$ if 
$\psi_-=0$ and $\psi_+$ is chosen sufficiently large. 

``SemismoothNewton_nonneg`` allows for an early termination of the Newton iteration by an a-posteriori error estimate based on the duality gap.

In [ ]:
weighted_data_space = L2(grid, weights = 1./(1.+data))
setting = RegularizationSetting(op=conv, penalty=L2, data_fid=weighted_data_space)
alpha = 0.0001

SSNewton_nn = SemismoothNewton_nonneg(setting, data, alpha, TOL = 0.01, 
                                    logging_level=logging.DEBUG, cg_logging_level=logging.INFO)
it = iter(SSNewton_nn)

Run this cell until no changes in the active set occur. 

In [ ]:
x,y = next(it)
act= SSNewton_nn.active
act_old = SSNewton_nn.active_old
lam = SSNewton_nn.lam
#act = SSNewton_bl.active_minus
#act_old = SSNewton_bl.active_minus_old
#lam = SSNewton_bl.lam_minus

plt.rcParams.update({'font.size': 12})
extent = [grid.axes[0][0],grid.axes[0][-1], grid.axes[1][0], grid.axes[1][-1]]
fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize = (18,6))

vmax = np.max(exact_sol)
mycmap = mplib.colormaps['hot']
mycmap.set_under((0,1,0,1.))
im1= ax1.imshow(x.T,extent=extent,origin='lower',vmax=vmax,vmin = -vmax/1000,
                    cmap= mycmap)
fig.colorbar(im1,extend='min')
ax1.title.set_text('reconstruction')

im2 = ax2.imshow(lam.T/alpha, extent=extent,origin='lower',vmin = -vmax/1000,
                 cmap= mycmap)
fig.colorbar(im2,extend='min')
ax2.title.set_text('Lagrange parameter')

im3 = ax3.imshow(2/3*act.T+1/3*act_old.T, extent=extent, origin='lower',vmin=0,vmax=1,cmap= mplib.colormaps['gnuplot2'])
ax3.title.set_text('active set')
fig.colorbar(im3)

### Choose $\alpha$ by discrepancy principle
To choose the regularization parameter $\alpha$ in a data-driven manner, we run constrained Tikhonov regularization on a logarithmically equidistant grid of alpha-values and apply the discrepancy principle. The noise level is estimated from the data using the fact that 
expectation and variance of Poisson distributed random variables coincide.

In [ ]:
weighted_data_space = L2(grid, weights = 1./(1.+data))
setting = RegularizationSetting(op=conv, penalty=L2, data_fid=weighted_data_space)
solver = SemismoothNewtonAlphaGrid(setting, data, (1e-3,1/2), logging_level=logging.DEBUG,tol_fac = 0)
stoprule =  (CountIterations(max_iterations=30)
   +Discrepancy(setting.h_codomain.norm, data,
        noiselevel=setting.h_codomain.norm(np.sqrt(data[:])), tau=1.05)
)
for step, [x,y] in enumerate(solver.while_(stoprule)):
    comparison_plot(grid,exact_sol,x,title_left='reconstruction SSNewton {}'.format(step),residual = data-y)
    plt.show()
